In [ ]:
import os
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import object_detection
import wget
import time
import shutil
from lxml import etree

# <<<---configuration--->>>

In [ ]:
steps = '2000'

# <<<------------------------>>>

making new name for custom detection model: 
(if u get error for first time - just skip and going)

In [ ]:
model_path = os.path.join('Tensorflow', 'workspace','my_models')
if not os.path.exists(model_path):
    current_model_name = "custom_detection_model_00"
else:
    models_names = [name for name in os.listdir(model_path) if os.path.isdir(model_path)]
    current_model_name = 'custom_detection_model_'+str(len(models_names))

In [ ]:
CUSTOM_MODEL_NAME = current_model_name 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'SCRIPTS_PATH': os.path.join('Tensorflow', 'workspace','my_models',CUSTOM_MODEL_NAME,'dev','scripts'),
    'DOWNLOAD_REPO_PATH': os.path.join('Tensorflow','tf_repo_download_models'),
    'APIMODEL_PATH': os.path.join('Tensorflow','tf_repo_download_models','apimodel'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','my_models',CUSTOM_MODEL_NAME), 
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','my_models',CUSTOM_MODEL_NAME,'dev','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','my_models',CUSTOM_MODEL_NAME,'dev','pre-trained-models'),
    'PRETRAINED_MODEL_PATH_DOWNLOAD': os.path.join('Tensorflow','tf_repo_download_models','pre-trained-models')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','my_models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['DOWNLOAD_REPO_PATH'],'scripts', TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

making label list:

In [ ]:
train_folder_path = os.path.join('Tensorflow','workspace','images','train')
tag = True
xml_files_name = []
adnotation = []
all_files_from_train_dir = [f for f in listdir(train_folder_path) if isfile(join(train_folder_path, f))] #lista nazw plikow
for i in all_files_from_train_dir :
    if i[-1]=='l': (xml_files_name.append(i))
xml_files_name.sort()

for xml in xml_files_name:
    XML_PATH = os.path.join(train_folder_path,xml)
    root = etree.parse(XML_PATH) #open xml file
    myroot = root.getroot() 
    label_name_from_xml = (myroot.find('.//name')).text #getting name from xml file
    if label_name_from_xml not in adnotation: # not save duplicates
        adnotation.append(label_name_from_xml)

labels = []
one_label={}
for x in range(len(adnotation)):
    one_label['name'] = adnotation[x]
    one_label['id'] = x+1
    temporary  = one_label.copy()
    labels.append(temporary)
    one_label.clear()
    
print("labels=[")
for x in labels:
    print(x)
print("]")

creating folders from path:

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

copy tarin and test folders with images to model folder:

In [ ]:
source_train_path = os.path.join(paths['IMAGE_PATH'],'train') 
source_test_path = os.path.join(paths['IMAGE_PATH'],'test')
destination_train_path = os.path.join(paths['CHECKPOINT_PATH'],'dev','img','train')
destination_test_path = os.path.join(paths['CHECKPOINT_PATH'],'dev','img','test')

shutil.copytree(source_train_path, destination_train_path) #copy folders method
shutil.copytree(source_test_path, destination_test_path) 

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
if not os.path.exists(os.path.join(paths['PRETRAINED_MODEL_PATH_DOWNLOAD'],PRETRAINED_MODEL_NAME)): 
    !wget {PRETRAINED_MODEL_URL}
    
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH_DOWNLOAD']}
    !cd {paths['PRETRAINED_MODEL_PATH_DOWNLOAD']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
#     !wget {PRETRAINED_MODEL_URL}
#     !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
#     !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    shutil.copytree(os.path.join(paths['PRETRAINED_MODEL_PATH_DOWNLOAD'],PRETRAINED_MODEL_NAME), os.path.join(paths['PRETRAINED_MODEL_PATH'],PRETRAINED_MODEL_NAME))
else: shutil.copytree(os.path.join(paths['PRETRAINED_MODEL_PATH_DOWNLOAD'],PRETRAINED_MODEL_NAME), os.path.join(paths['PRETRAINED_MODEL_PATH'],PRETRAINED_MODEL_NAME))

In [ ]:
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {os.path.join(paths['DOWNLOAD_REPO_PATH'],'scripts')}
    shutil.copy(files['TF_RECORD_SCRIPT'], paths['SCRIPTS_PATH'])
else: shutil.copy(files['TF_RECORD_SCRIPT'], paths['SCRIPTS_PATH'])

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

In [ ]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],steps)

In [ ]:
start = time.time()

!{command}

end = time.time()
hard_time =  (end-start)/60
print('\n Training time: {:.2f} min'.format(hard_time))

evaluation:

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

In [ ]:
!{command}